In [2]:
import pandas as pd
data = pd.read_csv('/kaggle/input/memes-dataset/Dataset/hateful_memes_original.csv', encoding= 'ISO-8859-1')
data.head(7)

,text,img,label
0,a school bus that was engulfed in flames,img/32674.png,0
1,when you ask your dad who is a retired drill s...,img/10246.png,1
2,how i see kim burell everytime she grabs a mic!,img/14570.png,1
3,doing o's with the smoke,img/05316.png,0
4,im gettin white girl wasted tonight,img/20936.png,0
5,when you forget to take out the trash and you ...,img/62019.png,0
6,just think of this as the day the bad music died,img/64137.png,0


In [4]:
import tensorflow as tf
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

# https://huggingface.co/Salesforce/blip-image-captioning-large
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

for index, row in data.iterrows():
    img = row.img
    root_path = "/kaggle/input/memes-dataset/Dataset/"
    raw_image = Image.open(root_path+str(img)).convert('RGB')
    
    # unconditional image captioning
    inputs = processor(raw_image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=40)
    caption = processor.decode(out[0], skip_special_tokens=True)
    data.at[index, 'Caption'] = caption

GPU is available


In [5]:
data.head(10)

,text,img,label,Caption
0,a school bus that was engulfed in flames,img/32674.png,0,there is a fire that was engulfed in flames
1,when you ask your dad who is a retired drill s...,img/10246.png,1,a black and white photo of a fence with a quot...
2,how i see kim burell everytime she grabs a mic!,img/14570.png,1,there is a gorilla standing on its back in the...
3,doing o's with the smoke,img/05316.png,0,there is a man in a red hoodie smoking a cigar...
4,im gettin white girl wasted tonight,img/20936.png,0,woman sitting on a couch with a plate of food ...
5,when you forget to take out the trash and you ...,img/62019.png,0,there is a boy holding a blue bag with a tooth...
6,just think of this as the day the bad music died,img/64137.png,0,a close up of a person with a gun and a captio...
7,hurry up and join us for play,img/51076.png,0,there are three children dressed in nativity c...
8,when mom asks where all the jews went,img/73021.png,0,there is a man with glasses and a beard holdin...
9,"white kid: ""i don't know how to shoot"" dad: ""j...",img/75381.png,1,there is a man and a boy that are holding a ba...


In [6]:
data.to_csv('data.csv', index=False)